Приближённое вычисление интегралов при помощи квадратурных формул Наивысшей Алгебраической Степени Точности (КФ НАСТ) и Интегральных Квадратурных Формул (ИКФ)

Импорт библиотек

In [6]:
import math
import numpy
import scipy.integrate as spi
import numpy as np
from scipy import integrate, optimize

Определение параметров

Задаются основные параметры: N - количество узлов квадратурной формулы, EPS - точность вычислений.

In [7]:
N = 10
EPS = 10 ** -12

Функция bisection_method(f, a, b)

Метод бисекции для нахождения корней функции f на интервале [a, b].

In [8]:
def bisection_method(f, a, b):
    roots = []
    intervals = []
    h = (b - a) / 100
    x1 = a
    x2 = x1 + h
    y1 = f(x1)
    while x2 <= b:
        y2 = f(x2)
        if y1 * y2 <= 0:
            intervals.append((x1, x2))
        x1 = x2
        x2 += h
        y1 = y2

    for a, b in intervals:
        while b - a > 2 * 10 ** (-13):
            c = (a + b) / 2
            if f(a) * f(c) <= 0:
                b = c
            else:
                a = c
        roots.append((a + b) / 2)
    l = len(roots)
    if l % 2 != 0:
        roots[int(l / 2)] = 0
    return roots

Полином степени N.

In [9]:
def poly(x):
    return sum(x ** j for j in range(N))

Полином степени 2N - 1.

In [10]:
def poly_2n(x):
    return x ** (2 * N - 1)

Функция f, заданная как sin(x).

In [11]:
def f(x):
    return math.sin(x)

Весовая функция p, заданная как exp(x).

In [12]:
def p(x):
    return math.exp(x)

Вычисление приближенного значения интеграла функции func на интервале [a, b] с использованием ИКФ с n узлами.

In [13]:
def ikf(func, a, b, n):
    h = (b - a) / n
    nodes = [a + i * h for i in range(n)]
    moments = [integrate.quad(lambda x: p(x) * x ** k, a, b)[0] for k in range(n)]
    coefficients = np.linalg.solve(np.rot90(np.vander(nodes)), moments)
    print("Моменты весовой функции:", moments)
    print("Найденные узлы:", nodes)
    print("Коэффициенты построенной ИКФ:", coefficients)
    approx_integral = sum(coeff * func(node) for node, coeff in zip(nodes, coefficients))
    return approx_integral

Вычисление приближенного значения интеграла функции func на интервале [a, b] с использованием КФ НАСТ с n узлами.

In [14]:
def kfnast(func, a, b, n):
    moments = [integrate.quad(lambda x: p(x) * x ** k, a, b)[0] for k in range(2 * n)]
    matr_left = np.array([[moments[j] for j in range(i, i + n)] for i in range(n)])
    a_list = np.linalg.solve(matr_left, [(-1) * moments[i] for i in range(n, 2 * n)])
    w_n = lambda x: x ** n + sum(a_list[i] * x ** i for i in range(n))
    nodes = bisection_method(w_n, a, b)
    print("Найденный ортогональный многочлен:", "x ** n + " + " + ".join(f"{a_list[i]} * x ** {i}" for i in range(n)))

    print([i.real for i in numpy.roots([1] + a_list[::-1])])
    coefficients = np.linalg.solve(np.rot90(np.vander(nodes)), moments[:n])
    print("Моменты весовой функции:", moments)
    print("Найденный ортогональный многочлен:", "x ** n + " + " + ".join(f"{a_list[i]} * x ** {i}" for i in range(n)))
    print("Найденные узлы:", nodes)
    print("Коэффициенты построенной КФНАСТ:", coefficients)
    approx_integral = sum(coeff * func(node) for node, coeff in zip(nodes, coefficients))
    return approx_integral


Основной блок программы

Здесь пользователь вводит границы интегрирования a и b, после чего программа проводит расчёты и выводит результаты.

In [15]:
print("Приближённое вычисление интегралов при помощи КФ НАСТ. Вариант 8. [𝑎, 𝑏] = [0, 1], 𝑓(𝑥) = sin(𝑥), 𝜌(𝑥) = 𝑒^x")
a, b = float(input("Введите границу интегрирования a: ")), float(input("Введите границу интегрирования b: "))

d = {f: "sin(x)", poly: "1 + " + " + ".join(f"x^{h}" for h in range(1, N)), poly_2n: f"x ** {2 * N - 1}"}
for i in [poly, f]:
    print("\n===================================")
    print("Функция:", d[i])
    real_val, _ = spi.quad(lambda x: i(x) * p(x), a, b)
    print("Значение интеграла по отрезку [a, b] в библиотеке scipy:", real_val)
    ikf_result = ikf(i, a, b, N)
    print(f"Приближенное значение ИКФ ({N} узлов): {ikf_result:.15f}")
    print(f"Абсолютная фактическая погрешность: {abs(ikf_result - real_val):.20f}")
    print("\n")
for j in [f, poly_2n]:
    print("Функция:", d[j])
    real_val, _ = spi.quad(lambda x: j(x) * p(x), a, b)
    kfnast_result = kfnast(j, a, b, N)
    print(f"Приближенное значение КФНАСТ ({N} узлов): {kfnast_result:.15f}")
    print("Абсолютная фактическая погрешность:", abs(kfnast_result - real_val))
    print("\n===================================")

Приближённое вычисление интегралов при помощи КФ НАСТ. Вариант 8. [𝑎, 𝑏] = [0, 1], 𝑓(𝑥) = sin(𝑥), 𝜌(𝑥) = 𝑒^x
Введите границу интегрирования a: 0
Введите границу интегрирования b: 2

Функция: 1 + x^1 + x^2 + x^3 + x^4 + x^5 + x^6 + x^7 + x^8 + x^9
Значение интеграла по отрезку [a, b] в библиотеке scipy: 1432.9663933134127
Моменты весовой функции: [6.3890560989306495, 8.389056098930649, 12.7781121978613, 20.7781121978613, 35.112448791445196, 60.887551208554804, 107.57428308023283, 192.77919910149348, 349.3647685142989, 638.9138060238032]
Найденные узлы: [0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1.0, 1.2000000000000002, 1.4000000000000001, 1.6, 1.8]
Коэффициенты построенной ИКФ: [ -0.33856495   4.35871714 -17.91774275  48.83265857 -84.56749471
 103.1360022  -85.6934416   51.0637742  -18.59193915   6.10708715]
Приближенное значение ИКФ (10 узлов): 1432.966393313411572
Абсолютная фактическая погрешность: 0.00000000000113686838



Функция: sin(x)
Значение интеграла по отрезку [a, b] в библиот